In [519]:
from docx import Document
import os
import re
import csv
import sys

### indent(para) Example
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Change of control means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Code shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [520]:
'''
Author: Yufei

Get the indent space of the paragraph, used in find_patterns and match function

-para = paragraph

return a num means the indent space or null if no indent
'''
def indent(para):
    return para.paragraph_format.first_line_indent

### bullet_list(para) Example
Change of control means that 

&nbsp;&nbsp;&nbsp;&nbsp;(a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or 

&nbsp;&nbsp;&nbsp;&nbsp;(b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.


In [521]:
'''
Author: Eden

Check if the paragraph is bullet ordered list, used in find_patterns and match function

-para = paragraph

return the regex pattern of this paragraph or None
'''
def bullet_list(para):
    pattern = None
    flag1 = re.match("\([A-Z]+\)", para.text.lstrip()[:10])
    flag2 = re.match("\([a-z]+\)", para.text.lstrip()[:10])
    flag3 = re.match("\([0-9]+\)", para.text.lstrip()[:10])
              
    if flag1:
        pattern = "\([A-Z]+\)"
    elif flag2:
        pattern = "\([a-z]+\)"
    elif flag3:
        pattern = "\([0-9]+\)"
    else:
        pattern = None
    return pattern

In [522]:
'''
Testing Function
'''
with open(out_path, 'ab') as f:
    # Input folder path
    directory_path = '/Users/yangyi/Desktop' 
    fileName = 'test.docx'
    document = Document(directory_path + "/" +fileName)
    
    for p in document.paragraphs:   
        pattern = bullet_list(p)
        print pattern
f.close()

\([0-9]+\)
\([a-z]+\)
None
None


### isListPara(para) Example
1. any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or 
2. a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [523]:
'''
Author: Alan

Check if the paragraph is list paragraph rather than normal paragraph, used in find_patterns and match function

-para = paragraph

return True/False
'''
def isListPara(para):
    paraFormat = para.style.name
    if paraFormat == 'List Paragraph':
        return True
    return False

### isNormal(para) Example
any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or 

In [524]:
'''
Author: Alan

Check if the paragraph is Normal rather than list paragraph，used in match function

-para = paragraph

return True/False
'''
def isNormal(para):
    paraFormat = para.style.name
    if paraFormat == 'Normal':
        return True
    return False

### isNormalFollwingListPara(paras, curIndex) Example
Change of control means that 
1. any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or 
2. a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [525]:
'''
Author: Alan

Check if the paragraph followed by list paragraph, used in find_patterns function

-paras = all the paragraph
-curIndex = the index of current paragraph

return True/False
'''
def isNormalFollowingListPara(paras, curIndex):
    targetFormat = paras[curIndex].style.name
    if curIndex == len(paras) - 1:
        return False
    nextFormat = paras[curIndex+1].style.name
    if nextFormat == 'List Paragraph' and targetFormat == 'Normal':
        return True
    return False

### bold(para) Example
**Change of control** means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

**Code** shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [526]:
'''
Author: Zheng

Check if the bold font in the paragraph, used in match function

-para = paragraph

return True/False
'''
def bold(para):
    paras_toPrint = []
    for run in para.runs:
        if run.bold:
            #print run.text,'bold'
            paras_toPrint.append(run.text)              
    if len(paras_toPrint) == 0:
        return False
    else:
        return True

### is_bold(para, keywords) Example
**Change of control** means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [527]:
'''
Author: Zheng

Check if the bold font and keyword in the paragraph, used in find_patterns function

-para = paragraph
-keywords = search words

return True/False
'''
def is_bold(para, keywords):
    paras_toPrint = []
    if keyword.lower() in para.text.lower(): 
        for run in para.runs:
            if run.bold:
                #print run.text,'bold'
                paras_toPrint.append(run.text)
                return True
    return False

### is_italic(para, keywords) Example
_Change of control_ means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [528]:
'''
Author: Zheng

Check if the italic font and keywords in the paragraph, used in match function

-para = paragraph

return True/False
'''
def is_italic(para, keywords):
    paras_toPrint = []
    if keywords.lower() in para.text.lower():
        for run in para.runs:
            if run.italic:
                paras_toPrint.append(run.text)
                return True
    return False

### italic(para) Example
_Code_ shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [529]:
'''
Author: Zheng

Check if the italic font in the paragraph, used in match function

-para = paragraph

return True/False
'''
def italic(para):
    paras_toPrint = []
    for run in para.runs:
        if run.italic:
            #print run.text,'italic'
            return True
            paras_toPrint.append(run.text)
    return False

### is_underline(para, keywords) Example
Change of control(Underline) means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [530]:
# Zheng
def is_underline(para, keywords):
    paras_toPrint = []    
    if keywords.lower() in para.text.lower():
        for run in para.runs:
            if run.underline:
                #print run.text,'underline'
                paras_toPrint.append(run.text)
    if len(paras_toPrint) == 0:
        return False
    else:
        return True

### underline(para) Example

Code(Underline) shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [531]:
'''
Author: Zheng

Check if the underline in the paragraph, used in match function

-para = paragraph

return True/False
'''
def underline(para):
    paras_toPrint = []
    for run in para.runs:
        if run.underline:
            return True
            #print run.text,'underline'
            paras_toPrint.append(run.text)
    return False

### is_double_quotes(para, keywords) Example:
"Change of control" means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [532]:
'''
Author: Zoe

Check if the keyword is double quoted in the paragraph, used in find_patterns function

-para = paragraph

return True/False
'''
def is_double_quotes(para, keywords):
    words = re.findall(r'\"(.+?)\"', para.text)
#     print '\tIS Double Quotes', words
    if not words:
        return False
    if para.text.startswith("\"" + words[0] + "\""):   #and words[0].lower() == keyword.lower():
        return True       
    else:
        return False

### double_quotes(para) Example:

"Code" shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.


In [533]:
'''
Author: Zoe

Check if the double quotes in the paragraph, used in match function

-para = paragraph

return True/False
'''
def double_quotes(para):
    """
        Find if paragraph starts with double quotes
        
        return: True or False
    """
    words = re.findall(r'(\".+?\")', para.text)
    if not words:
        return False
    if para.text.startswith(words[0]):
        return True       
    else:
        return False

### is_single_quotes(para, keywords) Example:
'Change of control' means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.


In [534]:
'''
Author: Zoe

Check if the keyword is single quoted in the paragraph, used in find_patterns function

-para = paragraph

return True/False
'''
def is_single_quotes(para, keywords):
    words = re.findall(r'\'(.+?)\'\s',para.text)
    if not words:
        return False
    if para.text.startswith("\'"+ words[0] + "\'"): # and words[0].lower() == keyword.lower():
        return True
    else:
        return False

### single_quotes(para) Example:

'Code' shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.


In [535]:
'''
Author: Zoe

Check if single quotes in the paragraph, used in match function

-para = paragraph

return True/False
'''
def single_quotes(para):
    words = re.findall(r'(\'.+?\')\s',para.text)
    if not words:
        return False
    if para.text.startswith(words[0]):
        return True
    else:
        return False

### is_upper_case(para, keywords) Example:
CHANGE OF CONTROL shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.


In [536]:
'''
Author: Zoe

Check if the keyword is upper case in the paragraph, used in find_patterns function

-para = paragraph

return True/False
'''
def is_upper_case(para, keywords):
    words = re.findall(r'([A-Z]+?)\s',para.text)
    if not words:
        return False
    if para.text.startswith(words[0]): # and words[0].lower() == keyword.lower():
        return True
    else:
        return False

### upper_case(para) Example:

UPPERCASE shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.


In [537]:
'''
Author: Zoe

Check if upper case word in the paragraph, used in match function

-para = paragraph

return True/False
'''
def upper_case(para):
    words = re.findall(r'([A-Z]+?)\s',para.text)
    if not words:
        return False
    if para.text.startswith(words[0]):
        return True
    else:
        return False

#### Remove_Stopwords Test

In [538]:
'''
Author: Eden

Read stopwords from files and return a stopwords set.

-filename = file name

return the set of stopwords
'''
def read_stop_words(filename):
    vector = []
    with open(filename, 'r') as f:
        input = f.read().splitlines()

    [vector.append(input[i]) for i in range(len(input))]   
    return set(vector)

In [539]:
'''
Author: Zoe

Remove the stopwords such as articles, prep and so on from keywords

-keywords = search word

return the string without stopwords
'''

def remove_stopwords(keywords, stop_set):
    word_list = re.split(r'\s',keywords)
    filtered_words = [word for word in word_list if word not in stop_set]
    words = " ".join(filtered_words)
    return words

In [507]:
'''
Testing Function
'''
directory_path = '/Users/yangyi/Desktop' 
fileName = 'stop_words.txt'
stop_set = read_stop_words(directory_path + '/' +  fileName)
keyword = "Change of Control"
print remove_stopwords(keyword, stop_set)

Change Control


In [540]:
'''
Author: Eden

Traversing through each of the paragraphs and find the paragraph containing the keywords

-paras = all paragraphs
-keywords = search words

return a list of paragraph index 
'''
def search_2(paras, keyword):
    index = []
    i = 0
    target = remove_stopwords(keyword.lower(), stop_set)
    while (i<len(paras)):
        p = paras[i]
        if contains_sliding_window(p.text.lower(), target, 2 * len(keyword)):
            print 'test3'
            index.append(i)
        i = i+1
    return index

In [541]:
'''
Author: Eden

Find the keywords by sliding windows so that even the order of keywords change, we can still fetch the results

-text = all paragraphs
-keywords = search words
-win_len = the length of sliding window

return True if the keyword is in text within a sliding window by random orders.
'''
def contains_sliding_window(text, keyword, win_len):
    target_words = re.split('[\s]', keyword)
    probe = target_words[0]
#     print probe
#     print text
    
    flag = False
    i = 0
    while (True):
        text = text[i:]
        if probe in text:
            i = text.index(probe)
            l_pointer = max(0,i - win_len)
            r_pointer = min(len(text),i + win_len)
            window_str = text[l_pointer:r_pointer]
            flag = True
            for target_word in target_words:
                if target_word not in window_str:
                    flag = False
            if flag:
                return True
        else:
            break

In [542]:
'''
Testing Function
'''
text = 'aaChange of Control means an event or series of events by which:'
if contains_sliding_window(text.lower(), 'change control', 2 * len(keyword)):
    print text + '1'

aaChange of Control means an event or series of events by which:1


### upper_camel_case(para, keywords) Example:
Capitalized Lease Obligations means, with respect to any Person, all outstanding obligations of such Person in respect of Capital Leases, taken at the capitalized amount thereof accounted for as indebtedness in accordance with GAAP.

Change of Control means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

Code shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [543]:
'''
Author: Zoe

Still Testing
'''
def upper_camel_case(para, stop_set):
    words_list = re.split(" ", para.text)
    new_para = remove_stopwords(para.text, stop_set)
    new_para_list = re.split(" ", new_para)
    new_words_list = re.findall(r'([A-Z][a-z0-9\']+)[^A-Z]?\s', new_para)
    last_word = ""
    for i in range(len(new_words_list)):
        if new_words_list[i] == new_para_list[i]:
            last_word = new_words_list[i]
        else:
            break
    result = []
    for j in range(len(words_list)):
        if last_word == words_list[j]:
            result.append(last_word)
            break
        else:
            result.append(words_list[j])
    if len(result) > 1:
        return True
    else:
        return False

### is_upper_camel_case() Example:
Change of Control means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.


In [554]:
'''
Author: Zoe

Still Testing
'''
def is_upper_camel_case(paragraph, keyword, stop_set):
    words_list = re.compile('\w+').findall(paragraph.text)
    new_paragraph = remove_stopwords(paragraph.text, stop_set)
    new_paragraph_list = re.compile('\w+').findall(new_paragraph)
    new_words_list = re.findall(r'([A-Z][a-z0-9\']+)[^A-Z]?\s', new_paragraph)

    last_word = ""
    for i in range(len(new_words_list)):
        if new_words_list[i] == new_paragraph_list[i]:
            last_word = new_words_list[i]
        else:
            break
    result = []
    for j in range(len(words_list)):
        if last_word == words_list[j]:
            result.append(last_word)
            break
        else:
            result.append(words_list[j])
    results = " ".join(result)
    if results.lower() == keyword.lower():
        return True
    else:
        return False

In [545]:
# s = is_upper_camel_case('''"Change of Control" means Other Connection Taxes that are imposed on or measured by net income (however denominated) or that are franchise Taxes or branch profits Taxes.
# ''', 'change of control')
# print s

In [546]:
'''
Author: Yufei

Traversing through each of the paragraphs and find the paragraph containing the keywords

-paras = all paragraphs
-keywords = search words

return a list of paragraph index 
'''
def search(paras, keywords):
    index = []
    i = 0
    while (i<len(paras)):
        p = paras[i]
        if keywords.lower() in p.text.lower():
            index.append(i)
        i = i+1
    return index

In [547]:
'''
Author: Yufei

Find the pattern of the paragraph containing the keywords by call the is_* function

-paras = all paragraphs
-index = paragraph index
-keyword = search words

return True/False list, 
[bold, italic, underline, double_quotes, single_quotes, upper case, (upper camel case), indent space, list paragraph, normal following list paragraph, bullet list]

'''
def find_patterns(paras, index, keyword, stop_set):
    paragraph = paras[index]
    patterns = []
    patterns.append(is_bold(paragraph, keyword))
    patterns.append(is_italic(paragraph, keyword))
    patterns.append(is_underline(paragraph, keyword))
    patterns.append(is_double_quotes(paragraph, keyword))
    patterns.append(is_single_quotes(paragraph, keyword))
    patterns.append(is_upper_case(paragraph, keyword))
    patterns.append(is_upper_camel_case(paragraph, keyword, stop_set))
    patterns.append(indent(paragraph))
    patterns.append(isListPara(paragraph))
    patterns.append(isNormalFollowingListPara(paras, index))
    patterns.append(bullet_list(paragraph))
    return patterns

In [548]:
'''
Author: Yufei

Find the pattern of the paragraph containing the keywords by call the is_* function

-paras = all paragraphs
-targetPatterns = the target pattern
-startIndex = paragraph index

return content
'''
def match(paras, targetPatterns, startIndex):
    
    i = startIndex + 1
    match = []
    
    print '\tTarget Patterns: ' + str(parttern_translate(targetPatterns))
    
    while (i<len(paras)):
        patterns = []
        p = paras[i]
        

        patterns.append(bold(p))
        patterns.append(italic(p))
        patterns.append(underline(p))
        patterns.append(double_quotes(p))
        patterns.append(single_quotes(p))
        patterns.append(upper_case(p))
        patterns.append(upper_camel_case(p, stop_set))
        patterns.append(indent(p))
        patterns.append(isListPara(p))
        patterns.append(isNormalFollowingListPara(paras, i))
        patterns.append(bullet_list(p))
        
        print '\n\tNext Paragraph:\t{0}\n\n\t\t{1}'.format(str(parttern_translate(patterns)), p.text.encode("utf-8"))
        
        if patterns == targetPatterns:
            print 'Bingo Bingo Bingo Bingo'
            break
        i = i + 1
    for x in range(startIndex, i):
        match.append(paras[x])
        
    return match

In [549]:
'''
Author: Eden

return the list of patterns' name

-pattern = the pattern list with true/false

return list of patterns name
'''
def parttern_translate(pattern):
    list = []
    if pattern[0]:
        list.append('blod')
    if pattern[1]:
        list.append('italic')
    if pattern[2]:
        list.append('underline')
    if pattern[3]:
        list.append('double_quotes')
    if pattern[4]:
        list.append('single_quotes')
    if pattern[5]:
        list.append('upper_case')
    if pattern[6]:
        list.append('upper_camel_case')
    if pattern[7]:
        list.append('indent_space')
    if pattern[8]:
        list.append('list_paragraph')
    if pattern[9]:
        list.append('normal_following_list_paragraph')
    if pattern[10]:
        list.append('bullet_list')
    return list
    

In [556]:

'''
Main Function.

02/17/2018 Eden
1. Modified the System output to make programmers easier to debug.
2. Modified the output format in .csv files
3. encode all output string to be utf-8
4. Added one function to output the patterns name rather than simple true/false
'''

########################################################################
out_filepath = '/Users/yangyi/Documents/CMU-S18/capstone/Capstone-Project-Mylan/output/'
out_filename = 'Output_search.csv'
out_path = out_filepath + out_filename
########################################################################


header = ['File Name', 'Keywords', 'Content'] # Column names in output file 

with open(out_path, 'wb') as f:
    wr = csv.writer(f)
    wr.writerow(header)
    
with open(out_path, 'ab') as f:
    # Input folder path
    directory_path = '/Users/yangyi/Documents/CMU-S18/capstone/Capstone-Project-Mylan/input' 
    
    stop_set = read_stop_words('/Users/yangyi/Documents/CMU-S18/capstone/Capstone-Project-Mylan/res/stop_words.txt')    
    
    # Parse through each file in the input folder path
    print 'Keyword: ' + 'Change of Control' + '\n'
    for fileName in os.listdir(directory_path):
        print '-------------------------------------------------------------------------'
        print 'File name: ' + fileName + '\n'
        
        # Create a Document object of each of the files
        document = Document(directory_path + "/" +fileName)
        # document = Document('/upper_camel_case+underline+after_list_paragraph.docx')
        
        
        # A list to add all the paragraphs in the document
        paras = []
        
        '''
        The following for loop is used to add all the paragraphs in the document to the paras list object
        '''
        for p in document.paragraphs:        
            paras.append(p)
            
        paras_number = len(paras) # Find the number of paragraphs in the document
        
        paras_toPrint = [] # A list to add all the paragraphs relevant to 'Change of Control' language in this case
        
        keyword = "Change of Control"
        keywordindex = search(paras, keyword)
        
#         print '\tkeywordIndex: ' + str(keywordindex)
#         print
        
        results = []
        result = ""
        for index in keywordindex:
            print '\tTarget Paragraph: [{0}]\n\t\t{1}\n'.format(index, paras[index].text.encode("utf-8"))
            target = find_patterns(paras, index, keyword, stop_set)
            
            if index == paras_number - 1:
                result = paras[index]
            else:
                result = match(paras, target, index)
            
            results.extend(result)

        wr = csv.writer(f)
        # Created a single list of all instances of 'Change of Control'
        # Ignored unrecognized ascii characters that throw an encoding error. Need improvement
        text =  [paras.text.encode('ascii','ignore') for paras in results]
        res = "\n".join(text)
        
        wr.writerow([fileName, keyword, res])
        
        print '\n'
print("end")
f.close()

Keyword: Change of Control

-------------------------------------------------------------------------
File name: after_list_paragraph.docx

	Target Paragraph: [0]
		“Change of Control” means an event or series of events by which:

	Target Patterns: ['underline', 'upper_camel_case', 'indent_space', 'normal_following_list_paragraph']

	Next Paragraph:	['underline', 'upper_camel_case', 'list_paragraph']

		any “person” or “group” (as such terms are used in Sections 13(d) and 14(d) of the Exchange Act, but excluding any employee benefit plan of such person or its subsidiaries, and any person or entity acting in its capacity as trustee, agent or other fiduciary or administrator of any such plan and any Initial Shareholder) becomes the “beneficial owner” (as defined in Rules 13d-3 and 13d-5 under the Exchange Act, except that a person or group shall be deemed to have “beneficial ownership” of all securities that such person or group has the right to acquire, whether such right is exercisable

In [349]:
os.getcwd()

'/Users/yangyi'

In [49]:
# Abhay
'''
Open the output file in append mode and pull related language from the files in the input folder
'''

########################################################################
out_filepath = 'Output/'
out_filename = 'Output_search.csv'
out_path = out_filepath + out_filename
########################################################################


header = ['File Name', 'Keywords', 'Change of Control'] # Column names in output file 

with open(out_path, 'wb') as f:
    wr = csv.writer(f)
    wr.writerow(header)

with open(out_path, 'ab') as f:
    # Input folder path
    directory_path = 'input' 
    
    # Parse through each file in the input folder path
    for fileName in os.listdir(directory_path):
        print fileName
        
        # Create a Document object of each of the files
        #document = Document(directory_path + "/" +fileName)
        document = Document('input/upper_camel_case+underline+after_list_paragraph.docx')
        
        
        # A list to add all the paragraphs in the document
        paras = []
        
        '''
        The following for loop is used to add all the paragraphs in the document to the paras list object
        '''
        for p in document.paragraphs:        
            paras.append(p)
            
        paras_number = len(paras) # Find the number of paragraphs in the document
        
        paras_toPrint = [] # A list to add all the paragraphs relevant to 'Change of Control' language in this case
        
        keyword = "Change of Control"
        keywordindex = search(paras, keyword)
        
        print('keywordIndex:', keywordindex)
        
        results = []
        result = ""
        for index in keywordindex:
            print 'Target Para', paras[index].text
            target = find_patterns(paras, index, keyword)
            
            if index == paras_number - 1:
                result = paras[index]
            else:
                result = match(paras, target, index)
            
            results.append(result)

        wr = csv.writer(f)
        wr.writerow([fileName, keyword, results])
        
print("end")
f.close()

IOError: [Errno 2] No such file or directory: 'Output/Output_search.csv'